In [1]:
# Import Modules
import os
import plaid
import requests
import datetime
import pendulum
import json
import pandas as pd
%matplotlib inline
# Ignore the SettingWithCopyWarning
pd.options.mode.chained_assignment = None

In [2]:
# Define JSON prettifier parameters
def pretty_print_response(response):
  print(json.dumps(response, indent=4, sort_keys=True))

In [3]:
# Read API keys in from txt files
file1 = open('./keys/plaid_client_id.txt','r')
plaid_client_id = file1.readline()
file1.close()

file2 = open('./keys/plaid_secret.txt','r')
plaid_secret = file2.readline()
file2.close()

file3 = open('./keys/plaid_public_key.txt','r')
plaid_public_key = file3.readline()
file3.close()

In [4]:
# Set Plaid environment variables
PLAID_CLIENT_ID = os.getenv('PLAID_CLIENT_ID', plaid_client_id)
PLAID_SECRET = os.getenv('PLAID_SECRET', plaid_secret)
PLAID_PUBLIC_KEY = os.getenv('PLAID_PUBLIC_KEY', plaid_public_key)
PLAID_ENV = os.getenv('PLAID_ENV', 'sandbox')
PLAID_PRODUCTS = os.getenv('PLAID_PRODUCTS', 'transactions')

# Plaid Access Token

In this section, you will use the plaid-python api to generate the correct authentication tokens to access data in the free developer Sandbox. This mimics how you might connect to your own account or a customer account, but due to privacy issues, this homework will only require connecting to and analyzing the fake data from the developer sandbox that Plaid provides. 

Complete the following steps to generate an access token:
1. Create a client to connect to plaid
2. Use the client to generate a public token and request the following items: 
['transactions', 'income', 'assets']
3. Exchange the public token for an access token
4. Test the access token by requesting and printing the available test accounts

### 1. Create a client to connect to plaid

In [5]:
# Set Plaid Institition ID
INSTITUTION_ID = "ins_109508"

In [6]:
# Set Plaid development environment
client = plaid.Client(client_id = PLAID_CLIENT_ID, 
                      secret=PLAID_SECRET,
                      public_key=PLAID_PUBLIC_KEY, 
                      environment=PLAID_ENV, 
                      api_version='2019-05-29')

### 2. Generate a public token

In [7]:
# create response from sandbox - detailed in plaid api documentation
create_response = client.Sandbox.public_token.create(
    INSTITUTION_ID, ['transactions'])

### 3. Exchange the public token for an access token

In [8]:
# Set access token from exchange response
response = client.Item.public_token.exchange(create_response['public_token'])
access_token = response['access_token']

### 4. Fetch Accounts

In [9]:
# Create call response for Accounts
accounts_response = client.Accounts.get(access_token)

In [10]:
# Fetch Accounts
accounts = accounts_response['accounts']
pretty_print_response(accounts[:3])

[
    {
        "account_id": "wNL3Mmaay4hQraGoKmm6Fny7X7gEyvfroKgmb",
        "balances": {
            "available": 100,
            "current": 110,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "0000",
        "name": "Plaid Checking",
        "official_name": "Plaid Gold Standard 0% Interest Checking",
        "subtype": "checking",
        "type": "depository"
    },
    {
        "account_id": "5rD7Pmee3WIk4vR73gg1uBWJxJGXWbcZ7jx8D",
        "balances": {
            "available": 200,
            "current": 210,
            "iso_currency_code": "USD",
            "limit": null,
            "unofficial_currency_code": null
        },
        "mask": "1111",
        "name": "Plaid Saving",
        "official_name": "Plaid Silver Standard 0.1% Interest Saving",
        "subtype": "savings",
        "type": "depository"
    },
    {
        "account_id": "J8ZkvDllA3hMpoDPzBBRt3qoxopwqkudgwbkr"

---

# Account Transactions with Plaid

In this section, you will use the Plaid Python SDK to connect to the Developer Sandbox account and grab a list of transactions. You will need to complete the following steps:


1. Use the access token to fetch the transactions for the last 90 days
2. Print the categories for each transaction type
3. Create a new DataFrame using the following fields from the JSON transaction data: `date, name, amount, category`. (For categories with more than one label, just use the first category label in the list)
4. Convert the data types to the appropriate types (i.e. datetimeindex for the date and float for the amount)

### 1. Fetch the Transactions for the last 90 days

In [11]:
# Set start date and end date for data
start_date = pendulum.now().subtract(days=90).to_date_string()
end_date = pendulum.now().to_date_string()

In [12]:
# Create a call response for transactions
transactions_response = client.Transactions.get(
    access_token,
    start_date = start_date,
    end_date = end_date,
)

In [13]:
# Fetch Transactions
transactions = transactions_response['transactions']
pretty_print_response(transactions[:3])

[
    {
        "account_id": "wNL3Mmaay4hQraGoKmm6Fny7X7gEyvfroKgmb",
        "account_owner": null,
        "amount": 89.4,
        "authorized_date": null,
        "category": [
            "Food and Drink",
            "Restaurants"
        ],
        "category_id": "13005000",
        "date": "2020-05-11",
        "iso_currency_code": "USD",
        "location": {
            "address": null,
            "city": null,
            "country": null,
            "lat": null,
            "lon": null,
            "postal_code": null,
            "region": null,
            "store_number": null
        },
        "name": "SparkFun",
        "payment_channel": "in store",
        "payment_meta": {
            "by_order_of": null,
            "payee": null,
            "payer": null,
            "payment_method": null,
            "payment_processor": null,
            "ppd_id": null,
            "reason": null,
            "reference_number": null
        },
        "pending": false,
     

### 2. Print the categories for each transaction

In [14]:
# Render data into DataFrame and print 'category' and 'transaction_id' columns
trns_df = pd.DataFrame(transactions)
trns_df[['category','transaction_id']]

,category,transaction_id
0,"[Food and Drink, Restaurants]",g8zpKWnnQRhwL5EnZbb1cmZjVzDa7msgndl7Q
1,"[Transfer, Credit]",nKN4anllgqIbdlGyZAABsp7kGg45vpF6Wg8M9
2,"[Travel, Airlines and Aviation Services]",yjodLnNNvPTpQN9EdqqZFKpvPq84aKhyXpr8R
3,"[Travel, Taxi]",8356eEMManUl6KwJqnnQtvKZdpQ7VvCwLexJj
4,"[Food and Drink, Restaurants]",9xlkQE114bIqMrN7LwwEikPRAvQrzkfRQMl9p
5,[Payment],6zVXjEQQxAh8doBbxrryiezXbm31qefgXnrPW
6,"[Food and Drink, Restaurants, Fast Food]",RrEBvDddk7IrR1QKxaaGS7k35AMKZ7hRb5zov
7,"[Shops, Sporting Goods]",v41BZnddmah7VdE19kkDFW4jbvJg5WcWkVoZD
8,"[Payment, Credit Card]",b15pvWkkNbh1yKGEpbbMCz9dxJya6zCVZrG6K
9,"[Travel, Taxi]",E1P7GDmmoqhjeDkx9VVZuqg7yE63AqFXk3j5k


### 3. Create a new DataFrame using the following fields from the JSON transaction data: date, name, amount, category. 

(For categories with more than one label, just use the first category label in the list)

In [15]:
# Render new DataFrame with 'date', 'name', 'amount', and 'category' columns
trns_df_cln = trns_df[['date','name','amount','category']]
trns_df_cln

,date,name,amount,category
0,2020-05-11,SparkFun,89.40,"[Food and Drink, Restaurants]"
1,2020-05-10,INTRST PYMNT,-4.22,"[Transfer, Credit]"
2,2020-04-30,United Airlines,500.00,"[Travel, Airlines and Aviation Services]"
3,2020-04-28,Uber,6.33,"[Travel, Taxi]"
4,2020-04-25,Tectra Inc,500.00,"[Food and Drink, Restaurants]"
5,2020-04-24,AUTOMATIC PAYMENT - THANK,2078.50,[Payment]
6,2020-04-24,KFC,500.00,"[Food and Drink, Restaurants, Fast Food]"
7,2020-04-24,Madison Bicycle Shop,500.00,"[Shops, Sporting Goods]"
8,2020-04-15,CREDIT CARD 3333 PAYMENT *//,25.00,"[Payment, Credit Card]"
9,2020-04-15,Uber,5.40,"[Travel, Taxi]"


### 4. Convert the data types to the appropriate types 

(i.e. datetimeindex for the date and float for the amount)

In [16]:
# Get data types of DataFrame
trns_df_cln.dtypes

date         object
name         object
amount      float64
category     object
dtype: object

In [17]:
# Convert 'category' column from list to string
trns_df_cln['category'] = trns_df_cln['category'].apply(', '.join)

In [20]:
# Convert 'date' column from string to datetime
trns_df_cln['date'] = pd.to_datetime(trns_df_cln['date'])
trns_df_cln.dtypes

date        datetime64[ns]
name                object
amount             float64
category            object
dtype: object

In [21]:
# Show that 'category' column is now string
trns_df_cln.head(5)

,date,name,amount,category
0,2020-05-11,SparkFun,89.40,"Food and Drink, Restaurants"
1,2020-05-10,INTRST PYMNT,-4.22,"Transfer, Credit"
2,2020-04-30,United Airlines,500.00,"Travel, Airlines and Aviation Services"
3,2020-04-28,Uber,6.33,"Travel, Taxi"
4,2020-04-25,Tectra Inc,500.00,"Food and Drink, Restaurants"


---

# Income Analysis with Plaid

In this section, you will use the Plaid Sandbox to complete the following:
1. Determine the previous year's gross income and print the results
2. Determine the current monthly income and print the results
3. Determine the projected yearly income and print the results

In [ ]:
# YOUR CODE HERE

---

# Budget Analysis
In this section, you will use the transactions DataFrame to analyze the customer's budget

1. Calculate the total spending per category and print the results (Hint: groupby or count transactions per category)
2. Generate a bar chart with the number of transactions for each category 
3. Calculate the expenses per month
4. Plot the total expenses per month

### Calculate the expenses per category

In [ ]:
# YOUR CODE HERE

### Calculate the expenses per month

In [ ]:
# YOUR CODE HERE